In [7]:
import cv2

#chimie
for i in range(6, 191):
    img = cv2.imread(f"temp/without_fingers/chimie/pagina_{i}_without_fingers.png")
    img_grayscale = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blur_img = cv2.GaussianBlur(img_grayscale, (7, 7), 0)
    thresh = cv2.threshold(blur_img, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (27,23))
    dilate = cv2.dilate(thresh, kernel, iterations=2)

    #find contours
    cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    cnts = sorted(cnts, key=lambda x: cv2.boundingRect(x)[0])

    grila_index1 = 1
    for c in cnts:
        x, y, w, h = cv2.boundingRect(c)
        if h > 150 and w > 150:
            roi = img[y:y+h, x:x+w]
            cv2.imwrite(f"temp/bbox/chimie/pagina_{i-5}_grila_{grila_index1}.png", roi)
            grila_index1 += 1
            cv2.rectangle(img, (x, y), (x + w, y + h), (36, 255, 12), 2)
            
    # cv2.imwrite(f"temp/bbox/chimie/pagina_{i-5}_bbox.png", img)
    print(f"Gata cu pagina_{i}_without_fingers.png")

#bio
# for i in range(7, 259):
#     img = cv2.imread(f"pdf2images/bio/pagina_{i}.png")
#     img_copy = img.copy()

#     img_grayscale = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#     blur_img = cv2.GaussianBlur(img_grayscale, (7, 7), 0)
#     thresh = cv2.threshold(blur_img, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
#     kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (27,23))
#     dilate = cv2.dilate(thresh, kernel, iterations=2)
#     # cv2.imwrite(f"temp/bbox/bio/pagina_{i-6}_DILATE.png", dilate)

#     #find contours
#     cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#     cnts = cnts[0] if len(cnts) == 2 else cnts[1]
#     cnts = sorted(cnts, key=lambda x: cv2.boundingRect(x)[0])

#     grila_index = 1
#     for c in cnts:
#         x, y, w, h = cv2.boundingRect(c)
#         if h > 150 and w > 75:
#             roi = img[y:y+h, x:x+w]
#             cv2.imwrite(f"temp/bbox/bio/pagina_{i-6}_grila_{grila_index}.png", roi)
#             grila_index += 1
#             cv2.rectangle(img, (x, y), (x + w, y + h), (36, 255, 12), 2)
            
#     # cv2.imwrite(f"temp/bbox/bio/pagina_{i-6}_bbox.png", img)
#     print(f"Gata cu pagina_{i}.png")


Gata cu pagina_6_without_fingers.png
Gata cu pagina_7_without_fingers.png
Gata cu pagina_8_without_fingers.png
Gata cu pagina_9_without_fingers.png
Gata cu pagina_10_without_fingers.png
Gata cu pagina_11_without_fingers.png
Gata cu pagina_12_without_fingers.png
Gata cu pagina_13_without_fingers.png
Gata cu pagina_14_without_fingers.png
Gata cu pagina_15_without_fingers.png
Gata cu pagina_16_without_fingers.png
Gata cu pagina_17_without_fingers.png
Gata cu pagina_18_without_fingers.png
Gata cu pagina_19_without_fingers.png
Gata cu pagina_20_without_fingers.png
Gata cu pagina_21_without_fingers.png
Gata cu pagina_22_without_fingers.png
Gata cu pagina_23_without_fingers.png
Gata cu pagina_24_without_fingers.png
Gata cu pagina_25_without_fingers.png
Gata cu pagina_26_without_fingers.png
Gata cu pagina_27_without_fingers.png
Gata cu pagina_28_without_fingers.png
Gata cu pagina_29_without_fingers.png
Gata cu pagina_30_without_fingers.png
Gata cu pagina_31_without_fingers.png
Gata cu pagina_3